<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/%EC%9E%A5%EA%B8%B0/T_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#T5-base-korean-text-summary

#T5는 Text-to-Text Transfer Transformer모델을 한국어와 영어 코퍼스를 이용하여 사전학습한 모델입니다.

출처 : lcw99/T5-base-korean-text-summary
https://github.com/Bomtori/pko_T5_fineTuning/blob/main/T_5.ipynb

In [ ]:
#파이썬 버전 확인
!python --version

Python 3.10.12


#필요 패키지 설치

In [ ]:
!pip install torch transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.4 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import T5TokenizerFast, T5ForConditionalGeneration, T5Config
from torch.utils.data import DataLoader, Dataset
import zipfile
import json
import pandas as pd
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 요약문 및 레포트 생성 데이터 파인 튜닝

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        source_text = item["passage"]
        target_text = item["generative_summary"]
        source_inputs = self.tokenizer.encode_plus(source_text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        target_inputs = self.tokenizer.encode_plus(target_text, padding="max_length", truncation=True, max_length=64, return_tensors="pt")
        return {
            "input_ids": source_inputs["input_ids"].squeeze(),
            "attention_mask": source_inputs["attention_mask"].squeeze(),
            "labels": target_inputs["input_ids"].squeeze(),
            "decoder_attention_mask": target_inputs["attention_mask"].squeeze()
        }


##### 요약문 데이터

In [ ]:
def load_dataset(train_zip_file_path, test_zip_file_path):
    def generator(fpath):
        with zipfile.ZipFile(fpath, "r") as fp:
            file_list = fp.namelist()
            file_list = filter(lambda x: x.endswith(".json"), file_list)
            for fname in file_list:
                item = json.load(fp.open(fname, "r"))
                ma = item["Meta(Acqusition)"]
                mr = item["Meta(Refine)"]
                ann = item["Annotation"]

                ann_dir = fname.split('/')[-2]

                generative_summary = ann["summary1"]
                if ann_dir == "20per":
                    extractive_summary = ann["summary3"]
                else:
                    extractive_summary = ann["summary2"]

                yield {
                    "doc_type": ma["doc_type"],
                    "passage_id": mr["passage_id"],
                    "passage": mr["passage"],
                    "generative_summary": generative_summary,
                    "extractive_summary": extractive_summary,
                }

    train_dataset_generator = generator(train_zip_file_path)
    test_dataset_generator = generator(test_zip_file_path)

    train_dataset = list(train_dataset_generator)
    test_dataset = list(test_dataset_generator)

    return train_dataset, test_dataset

In [ ]:
# 데이터셋 경로
train_zip_file_path = "/content/drive/MyDrive/data/022.요약문 및 레포트 생성 데이터/01.데이터/1.Training/라벨링데이터/TL1.zip"
test_zip_file_path = "/content/drive/MyDrive/data/022.요약문 및 레포트 생성 데이터/01.데이터/2.Validation/라벨링데이터/VL1.zip"


In [ ]:
# 데이터셋 로드
train_dataset, test_dataset = load_dataset(train_zip_file_path, test_zip_file_path)


##### 자체 제작 데이터

In [14]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/train.csv', encoding= 'cp949')
test = pd.read_csv('/content/drive/MyDrive/test.csv', encoding= 'cp949')

train

,news,summary
0,"“제가 만일 일한합병 때 나서 있었다면, 이완용이, 이용구, 송병준이 그런 놈들을...",내가 미국인과 걸어오는 춘자와 딱 마주치고 주춤하는 사이 춘자는 고개를 꽃꽃이 쳐들...
1,내가 퍽 어리석은 녀석이지요 ── 무척 못난 녀석이지요. 그렇지마는 지 금 와서 ...,학비 주시는 은인이나 교회 여러 직에 계신 분들의 추천을 얻느라 고심한 결과 동경에...
2,술을 마시지 않는 자에겐 이 약수터가 훌륭한 피서지이었지만 술을 마시 는 사람에겐...,내가 술을 제대로 계속하는 건강한 몸이었다면 이 약수터가 피서지로서의 대상이 되었을...
3,그러나 한 주일쯤 뒤에 정희는 동경 땅을 밟게 되었다. 이번의 정희의 동 경행에 ...,그러나 일주일 뒤에 정희는 약혼자인 남영식의 극력 찬성으로 동경에 가게 되었는데 그...
4,그 여자는 매우 고맙다고 인사를 하고 그제서야 매우 친절하게 굴며 몸을 녹이라고 화...,사냥군은 깜짝 놀라 벗어놓았던 망태를 둘러멘 뒤 총도 거꾸로 들고 뒷문으로 도망을 쳤다.
...,...,...
9899,S# 28. 동만네 집 전경(밤) 동만 모의 진한 핏빛 울음소리 들리는데 빗속에 누...,장맛비가 계속 오는 가운데 전쟁으로 외할머니의 아들 길준에 집안 분위기가 좋지 않다.
9900,S# 91. 염전 주막 방 안 천가가 문을 열어 주면 송화가 방으로 들어가 앉는다....,"송화라는 여인이 동호라는 남자와 오랜만에 재회하고, 그의 북소리에 맞춰 노래를 부른..."
9901,30대 중반의 정원은 불치의 병을 앓고 있는 사진사이다. 이제는 죽음을 담담하게 받...,"정원은 사진관에서 다림의 사진을 찍고, 다림은 화장품과 옷을 고르다가 포기한다. 다..."
9902,"S# 57. 수하 하숙방 (휴일 한낮) 수하, 은희가 준 엘피 음반을 정성껏 닦아 ...","홍연이 남자 선생님인 수하에게 반하고, 그의 팔을 꼬집은 일을 잊지 못하고 일기에 ..."


In [39]:
# 문자열이 아닌 데이터 유형을 선택
non_string_columns = train.select_dtypes(exclude='object')

# 문자열이 아닌 열을 확인하여 데이터 프레임에 문자열이 아닌 데이터가 있는지 확인
if non_string_columns.empty:
    print("데이터 프레임에 문자열이 아닌 데이터가 없습니다.")
else:
    print("데이터 프레임에 문자열이 아닌 데이터가 있습니다.")
    print("문자열이 아닌 열:")
    print(non_string_columns)

데이터 프레임에 문자열이 아닌 데이터가 없습니다.


In [17]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        source_text = item['news']
        target_text = item['summary']
        source_inputs = self.tokenizer.encode_plus(source_text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
        target_inputs = self.tokenizer.encode_plus(target_text, padding="max_length", truncation=True, max_length=64, return_tensors="pt")
        return {
            "input_ids": source_inputs["input_ids"].squeeze(),
            "attention_mask": source_inputs["attention_mask"].squeeze(),
            "labels": target_inputs["input_ids"].squeeze(),
            "decoder_attention_mask": target_inputs["attention_mask"].squeeze()
        }


In [40]:
def load_dataset(train_file_path, test_file_path):
    def generator(fpath):
        csv_file = pd.read_csv(fpath, encoding='cp949')
        csv_file = csv_file.dropna()
        for n, s in zip(csv_file['news'], csv_file['summary']):
            news = n
            summary = s
            yield {
                "news": news,
                "summary": summary
            }

    train_dataset_generator = generator(train_file_path)
    test_dataset_generator = generator(test_file_path)

    train_dataset = list(train_dataset_generator)
    test_dataset = list(test_dataset_generator)

    return train_dataset, test_dataset

In [41]:
train_file_path = '/content/drive/MyDrive/train.csv'
test_file_path = '/content/drive/MyDrive/test.csv'
train_dataset, test_dataset = load_dataset(train_file_path, test_file_path)

In [36]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration
import torch.optim as optim

# 파인튜닝한 모델 설정
model_path = "/content/drive/MyDrive/data/finetuning_model"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5TokenizerFast.from_pretrained(model_path, is_split_into_words=True)

# # 모델 및 토크나이저 설정
# tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-small')
# model = T5ForConditionalGeneration.from_pretrained('paust/pko-t5-small')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 1
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [43]:
# 데이터셋 전처리 및 데이터로더 설정
train_custom_dataset = CustomDataset(train_dataset, tokenizer)
train_dataloader = DataLoader(train_custom_dataset, batch_size=16, shuffle=True)

In [44]:
model.train()
for epoch in range(epochs):
    total_loss = 0
    with tqdm(total=len(train_dataloader), desc=f"Epoch {epoch+1}/{epochs}", unit="batch") as pbar:
        for batch in train_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            decoder_attention_mask = batch["decoder_attention_mask"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            pbar.set_postfix(loss=total_loss / (pbar.n + 1))
            pbar.update()

    print(f"Epoch: {epoch+1}, Loss: {total_loss / len(train_dataloader)}")


Epoch 1/1: 100%|██████████| 619/619 [07:04<00:00,  1.46batch/s, loss=0.65]

Epoch: 1, Loss: 0.6500597643832976


In [ ]:
# 파인 튜닝된 모델 저장
output_dir = "/content/drive/MyDrive/data/finetuning_model_sum"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# 모델 설정

In [ ]:
# 모델 및 토크나이저 설정
tokenizer = T5TokenizerFast.from_pretrained('paust/pko-t5-small')
model = T5ForConditionalGeneration.from_pretrained('paust/pko-t5-small')


In [ ]:
# GPU 사용 여부 확인 및 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# 데이터셋 전처리 및 데이터로더 설정
train_custom_dataset = CustomDataset(train_dataset, tokenizer)
train_dataloader = DataLoader(train_custom_dataset, batch_size=16, shuffle=True)


In [ ]:
import torch.optim as optim

# 파인 튜닝을 위한 하이퍼파라미터 설정
epochs = 3
learning_rate = 1e-4

# 선택한 최적화 알고리즘과 모델의 매개변수를 전달하여 optimizer를 초기화합니다.
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 학습

In [ ]:
model.train()
for epoch in range(epochs):
    total_loss = 0
    with tqdm(total=len(train_dataloader), desc=f"Epoch {epoch+1}/{epochs}", unit="batch") as pbar:
        for batch in train_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            decoder_attention_mask = batch["decoder_attention_mask"].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels, decoder_attention_mask=decoder_attention_mask)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

            pbar.set_postfix(loss=total_loss / (pbar.n + 1))
            pbar.update()

    print(f"Epoch: {epoch+1}, Loss: {total_loss / len(train_dataloader)}")


In [ ]:
from keras.models import load_model

model.save('pko_t5_finetune')

# 파인 튜닝 모델 저장

In [45]:
# 파인 튜닝된 모델 저장
output_dir = "/content/drive/MyDrive/data/finetuning_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('/content/drive/MyDrive/data/finetuning_model/tokenizer_config.json',
 '/content/drive/MyDrive/data/finetuning_model/special_tokens_map.json',
 '/content/drive/MyDrive/data/finetuning_model/tokenizer.json')

#파인튜닝 후 모델 및 토크나이저 로드

In [ ]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration

model_path = "/content/drive/MyDrive/data/finetuning_model"  # 저장된 모델의 디렉토리 경로

tokenizer = T5TokenizerFast.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)


#모델 사용

In [ ]:
input_text = """현대자동차 생산 공장에 처음으로 여성 직원이 투입될 전망입니다.
현대차는 올해 기술직 신규 채용 인원 400명 가운데 1차 최종합격자인 200명을 선정해 당사자에게 개별 통보했는데, 이 200명 가운데 6명이 여성인 걸로 나타났습니다.
금속노조 여성위원회는 어제 입장문을 내고 "회사 창립 후 여성 노동자에게 처음으로 열린 기술직 공채의 문"이라면서도 "합격자 200명 중 여성은 6명, 3%에 그쳤다"며 "더 많은 여성 노동자가 배제 없이 채용되길 바란다"고 밝혔습니다.
다만 이 6명은 현재 합격자로 통보받은 상황이고 최종 채용된 건 아닙니다.
앞서 현대차는 지난해 노조와 합의에 따라 국내 생산공장에서 근무할 기술직 직원을 10년 만에 채용하기로 했습니다.
올해 400명, 내년 300명으로 모두 700명 규모로, 올해 1차 채용 서류 접수 첫날에는 접속자가 몰려 홈페이지가 마비되는 일이
벌어지기도 했습니다."""  # 요약할 입력 문장

input_ids = tokenizer.encode(input_text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
outputs = model.generate(input_ids, max_length=300)
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Summary:", generated_summary)


In [ ]:
input_text = """러시아가 우크라이나를 침공한 지 500일이 넘었지만, 여전히 주요 소셜 미디어 플랫폼에선 우크라이나 전쟁과 관련해 잘못된 정보와 오해의 소지가 있는 게시물이 계속해서 확산하고 있다.
가장 널리 공유된 예 중 일부는 트위터에서 찾아볼 수 있었는데, 파란색 체크 표시가 있는 트위터 계정들이 게시한 것들이었다. ‘트위터 블루’라고 불리는 이 파란색 체크 표시는 다른 사용자에게 자신의 콘텐츠를 홍보하기 위해 돈을 지불하고 유료로 서비스를 구독하는 사람들이 받을 수 있다.
우크라이나의 무기는 프랑스 폭력 시위에 사용되지 않았다
최근 프랑스 폭동에 대해 오해의 소지가 있는 많은 게시물이 온라인상에서 공유됐다. 하지만 지난주 우크라이나에 대한 미국의 군사 지원에 초점을 맞춘 한 게시물이 크게 화제가 됐다.
해당 게시물은 두 개의 소총 사진이 있는 뉴스 웹사이트의 헤드라인 화면을 스크린샷해 올렸다.
헤드라인에는 “프랑스 경찰은 우크라이나에서 왔을지도 모르는 미국산 소총으로 공격받고 있다”고 쓰여 있다.
 “프랑스 경찰은 우크라이나에서 왔을지도 모르는 미국산 소총으로 공격받고 있다”고 쓰여 있는 트위터 화면. 거짓 정보다.
사진 출처,TWITTER
파란색 체크 표시를 가진 다른 트위터 계정들이 이 게시물을 공유하면서, 이 게시물은 결국 100만이 넘는 조회수를 기록했다.
BBC의 사실 검증 탐사보도 팀인 ‘BBC Verify’는 이를 검증하기 위해 텔레그램 메신저 앱의 친 크렘린궁 채널까지 추적한 결과 해당 게시물에 사용된 사진이 2012년 한 러시아 군사 관련 블로그에 등장하는 것을 발견했다. 모스크바 인근 사격장에서 열린 사격 대회에 대한 내용이었다.
BBC는 또한 위와 같은 헤드라인과 사진이 있는 온라인 기사를 찾을 수 없었으며, 미국이 우크라이나에 제공한 무기가 최근 프랑스의 소요 사태 동안 사용됐다는 증거도 존재하지 않는다.
프랑스 폭동: 온라인에 유포되는 폭동 루머
2023년 7월 3일
틱톡에 확산하는 우크라이나 전쟁 가짜 영상
2022년 4월 28일
트위터에서 활개치는 러시아 트롤
2023년 4월 14일
우크라이나 '아기 공장'에 대한 증거는 없다
파란색 체크 표시가 있는 몇몇 트위터 계정은 최근 러시아가 우크라이나에서 “아기 공장”을 발견했다는 주장을 홍보했다.
2세에서 7세 사이의 아이들이 “공장에서 생산되어”, “아동 성매매촌”으로 보내지거나 장기 매매를 위해 서양에 판매된다는 주장이다.
최근 러시아가 우크라이나에서 “아기 공장”을 발견했다는 주장을 담은 트위터 화면. 증거가 없는 주장이다.
사진 출처,TWITTER
BBC Verify가 해당 주장의 기원을 추적한 결과, 지난 3월 ‘더 피플스 보이스(The People’s Voice)’라고도 알려진 ‘유어뉴스와이어(YourNewsWire)’에 게재된 기사라는 것을 확인했다. 해당 단체는 팩트 체크 기관들이 인터넷에서 가짜 뉴스를 가장 많이 생산하는 업체 중 하나로 묘사해 온 곳이다.
이 단체는 과거 여러 허위 정보와 오해의 소지가 있는 이야기들을 홍보한 적이 있으며, 특히 지난 2017년 라스베이거스 총기 난사 사건에 대한 거짓 주장과 코로나19 백신 음모론 등을 퍼뜨린 바 있다.
러시아 정부와 크렘린궁이 통제하는 언론은 우크라이나에서의 불법 장기 매매에 대한 근거 없는 주장을 홍보한 전력이 있다.
크라마토르스크의 미사일 공격은 우크라이나가 한 것이 아니다
지난 6월 말, 우크라이나 동부 크라마토르스크 중심부에서 러시아의 미사일 공격으로 8명이 사망했다.
해당 공습 직후 트위터 블루를 구독하는 한 계정이 자신을 적법한 뉴스 출처라고 주장하면서 트위터에 한 게시글을 올렸다. 해당 게시글은 이번 공습이 우크라이나군에 의해 실수로 이뤄진 것이라며 나토군과 외국 용병들이 있는 군 막사도 공격받았다고 주장했다.
러시아 미사일 우크라 동부 중심부 강타
2023년 6월 28일
크라마토르스크 미사일 공습이 우크라이나군에 의한 것이라는 주장의 트위터 화면. 거짓이다.
사진 출처,TWITTER
이 게시글은 “스톰 쉐도우 미사일이 갑자기 궤적을 극적으로 바꿔 크라마토르스크를 타격했으며 외국 군인과 용병이 있는 우크라이나 군 막사를 궤멸시켰다”고 주장했다.
해당 게시글은 100만 회 이상 조회됐다.
그러나 우크라이나군이 발사한 미사일이 원인이라는 증거도, 군 막사를 타격했다는 증거도 전혀 없다.
젤렌스키 대통령은 선거를 취소하지 않았다
볼로디미르 젤렌스키 우크라이나 대통령이 우크라이나 선거를 “취소”했다는 주장이 최근 트위터에서 확산하고 있다.
그 증거로 사용자들은 지난 6월 BBC와의 인터뷰에서 젤렌스키 대통령이 했던 발언을 인용했다.
내년에 우크라이나에서 선거가 있을 것인지에 대한 질문에 젤렌스키 대통령은 이렇게 답했다. “우리가 [전쟁에서] 승리한다면 선거는 있을 것입니다. 즉 계엄령도, 전쟁도 없다는 것을 의미하죠. 법에 따르면, 전쟁이 없는 평시에 선거는 반드시 치러져야 합니다.”
우크라이나에 대한 미국의 원조에 비판적이었던 전 폭스 뉴스 진행자 터커 칼슨은 최근 시작한 그의 트위터 쇼에서 젤렌스키의 발언은 그가 우크라이나의 민주주의를 종식시켰다는 것을 증명했다고 말했다.
트위터 블루 계정들은 비슷한 주제의 내용을 수십만 번 이상 공유했다.
볼로디미르 젤렌스키 우크라이나 대통령이 우크라이나 선거를 “취소”하려고 협박한다는 주장의 트위터. 오해의 소지가 있다.
사진 출처,TWITTER
우크라이나 헌법은 계엄령 기간 중 의회 해산과 총선을 금지하고 있다. 즉 계엄 기간이 끝날 때까지 현 대통령과 의회가 책임을 유지한다는 뜻이다.
최근 올렉시 다닐로우 우크라이나 국가안보국방위원회 서기는 우크라이나 헌법에 따라 계염령이 시행되는 동안 “어떤 선거도 실시할 수 없다”고 공식 입장을 밝혔다.
한편 이 기사에서 언급한 허위 및 오해의 소지가 있는 트위터 블루 게시물에 관해 BBC Verify가 트위터 측에 연락을 취했으나, 트위터의 언론 담당 부서는 BBC의 문의를 받은 것은 인정했으나 관련한 답변은 거부했다."""  # 요약할 입력 문장

input_ids = tokenizer.encode(input_text, padding="max_length", truncation=True, max_length=1024, return_tensors="pt")
outputs = model.generate(input_ids, max_length=500)
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Summary:", generated_summary)


In [ ]:
input_text = """보건복지부는 11일 의약품 관련 기관·단체들과 간담회를 열고 의약품 부족과 일반의약품 가격 인상 문제 등을 논의했다고 밝혔다.

박민수 제2차관이 주재한 이날 간담회엔 식품의약품안전처, 대한의사협회, 대한약사회, 한국의약품유통협회, 한국제약바이오협회, 한국병원약사회, 의약품관리종합정보센터 관계자 등이 참석했다.

이 자리에서는 의약품 부족 문제에 대응하기 위해 지난 3월 구성된 '의약품 수급 불안정 민관 대응 협의체'의 진행 상황과 함께 현장에서 체감하는 의약품 수급 불안 현황과 원인, 제도 개선방안 등에 대해 논의했다.

또 처방전 없이 살 수 있는 일반의약품(OTC)의 최근 잇단 가격 인상과 관련해 가격 인상이 국민에게 큰 부담이 되지 않도록 업체의 자체적인 노력을 요청했다고 복지부는 전했다.

박 차관은 "앞으로 현장과 전문가 의견을 충실히 수렴해 의약품 부족 상황에 대한 면밀한 분석과 함께 부족 우려 의약품의 공급·수요 관리를 위한 제도개선 및 정보시스템 정비 등 체계적 대응 방안을 마련해 나가도록 하겠다"고 말했다.

한편 복지부는 공장 해외이전으로 지난해 3월 생산이 중단된 해열제 타이레놀 일부 제품(어린이용 타이레놀정 80mg, 타이레놀정 160mg)에 대해선 "이미 생산된 재고량이 상당량 존재하고 재허가를 시도하는 품목이 있어 상황을 살펴보고 있다"고 설명했다.

이들 제품은 편의점에서도 판매할 수 있는 안전상비의약품 13개에 포함돼 있는데, 생산 중단 이후 공급이 원활하지 않아 편의점 구입이 어렵다는 지적이 나온 바 있다.

복지부는 이들 2개 품목의 경우 "어린이부루펜시럽, 어린이용타이레놀현탁액 등 안전상비의약품 대체 품목이 존재하고 있다"며 "안전상비의약품 지정 취소 및 대체약 추가 지정 필요성은 하반기 내에 신속히 논의해 결정하겠다"고 밝혔다."""  # 요약할 입력 문장

input_ids = tokenizer.encode(input_text, padding="max_length", truncation=True, max_length=1024, return_tensors="pt")
outputs = model.generate(input_ids, max_length=512)
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Summary:", generated_summary)


In [ ]:
input_text = """생성형 AI 시대, 우리는 무엇을 준비해야 하는가과거 ‘대답’을 잘하는 사람이 전문가였던 시절에서 ‘검색’을 잘하는 사람이 전문가였던 시대를 지나, ‘질문’을 잘하는 사람이 전문가로 인정받는 세상을 맞이하고 있다. 생성형 AI는 우리가 정보를 다루는 방식에 큰 변화를 가져올 것이다. 그동안 우리는 필요한 정보를 신속하게 획득할 수 있는 검색 능력에 주로 의존해 왔으나, 앞으로는 필요한 답변을 원하는 양식대로 빠르게 얻어 내는 능력이 중요해질 것이다. 결국 AI에 일을 더 잘 시키기 위한 프롬프트를 찾는 작업, 이른바 프롬프트 엔지니어링(Prompt Engineering)이 중요해질 것이다. 일부 테크 기업들은 이미 프롬프트 엔지니어 채용 공고를 내기 시작했다. 2월 초 구글이 5,000억 원을 투자한 AI 스타트업 앤스로픽(Anthropic)은 지난달 연봉 3억~4억 원 수준의 프롬프트 엔지니어·데이터 라이브러리 관리자 채용 공고를 냈다. """  # 요약할 입력 문장

input_ids = tokenizer.encode(input_text, padding="max_length", truncation=True, max_length=1024, return_tensors="pt")
outputs = model.generate(input_ids, max_length=512)
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Summary:", generated_summary)


In [ ]:
input_text = """7월 초순까지 반도체와 대(對)중국 수출의 부진이 이어지며 전체 수출도 1년 전보다 15%가량 줄어들었다.

지난달 흑자로 돌아섰던 무역수지는 이달 초순까지 적자를 기록하며 올해 누적 무역적자는 290억달러에 육박한 것이다.

11일 관세청에 따르면 7월 1∼10일 수출액(통관 기준 잠정치)은 132억6천700만달러로 작년 같은 기간보다 14.8% 감소했다.

조업일수를 고려한 일평균 수출액도 14.8% 감소했고 이 기간 조업일수는 7.0일로 작년(7.0일)과 같았다.

월간 수출액은 전년 동월 대비 기준 작년 10월부터 지난달까지 9개월 연속 감소세를 이어가고 있다.

품목별로 보면 반도체 수출이 1년 전보다 36.8% 줄었으며 반도체 수출 감소는 월간 기준 지난달까지 11개월 연속 이어지고 있다.

철강제품(-14.5%), 석유제품(-51.3%), 무선통신기기(-27.1%) 등의 수출도 1년 전보다 줄었다.

반면 승용차(25.2%), 선박(74.0%) 등은 늘었다.

국가별로는 최대 교역국인 중국에 대한 수출이 20.6% 감소했고 대중 수출의 감소세는 13개월 넘게 지속되고 있다.

미국(-9.0%), 베트남(-32.5%), 일본(-20.8%) 등도 줄었으며 유럽연합(EU·22.4%), 인도(11.1%) 등은 늘었다.

이달 10일까지 수입액은 155억4천300만달러로 26.9% 감소했다.

3대 에너지원인 원유(-55.2%), 가스(-32.2%), 석탄(-59.0%) 등의 수입이 모두 줄었고 반도체(-23.8%), 석유제품(-33.5%), 반도체 제조장비(-14.7%) 등도 감소했다.

무선통신기기(48.1%)와 기계류(0.5%) 등은 늘었다.

국가별로는 베트남(13.7%), 말레이시아(6.1%) 등은 늘고 중국(-16.8%), 미국(-17.5%), EU(-5.9%) 등은 줄었다.

이달 1∼10일 무역수지는 22억7천600만달러 적자를 기록했다.

특히 지난달 같은 기간(14억2천700만달러 적자)보다 적자 규모가 늘었다.

무역수지는 월간 기준으로 작년 3월부터 지난달까지 15개월 연속 적자가 지속되다가 지난달 흑자로 돌아선 바 있다.

올해 들어 누적된 무역적자는 287억4천100만달러로 집계됐으며 이달 초순에 적자를 기록하며 올해 누적 적자 규모가 지난달 기준보다 늘었다.

이달 1∼10일 중국과의 무역수지는 8억4천300만달러 적자였다.

대중 무역적자는 작년 10월부터 9개월째 이어지고 있다.

"""  # 요약할 입력 문장

input_ids = tokenizer.encode(input_text, padding="max_length", truncation=True, max_length=1024, return_tensors="pt")
outputs = model.generate(input_ids, max_length=512)
generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Summary:", generated_summary)


#문서 요약 텍스트 파인튜닝

In [ ]:
# 데이터셋 경로
train_zip_file_path = "/content/drive/MyDrive/data/문서요약 텍스트/Training/신문기사_train_original.zip"
test_zip_file_path = "/content/drive/MyDrive/data/문서요약 텍스트/Validation/신문기사_valid_original.zip"


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, json_file, tokenizer):
        self.data = self.load_json(json_file)
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
      document = self.data[idx]
      sentences = [item[0]['sentence'] for item in document['text'] if len(item) > 0]
      target_text = document['abstractive'][0]

      input_text = " ".join(sentences)

      input_ids = self.tokenizer.encode(input_text, truncation=True, padding='max_length', max_length=512)
      target_ids = self.tokenizer.encode(target_text, truncation=True, padding='max_length', max_length=128)

      return {
          'input_ids': torch.tensor(input_ids, dtype=torch.long),
          'attention_mask': torch.tensor([1] * len(input_ids), dtype=torch.long),
          'target_ids': torch.tensor(target_ids, dtype=torch.long),
          'target_attention_mask': torch.tensor([1] * len(target_ids), dtype=torch.long)
      }



    def load_json(self, json_file):
        with open(json_file, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data['documents']


In [ ]:
# JSON 파일 압축 해제 및 데이터 로드
train_zip_file_path = "/content/drive/MyDrive/data/문서요약 텍스트/Training/신문기사_train_original.zip"
test_zip_file_path = "/content/drive/MyDrive/data/문서요약 텍스트/Validation/신문기사_valid_original.zip"

train_extracted_path = "/content/train"
test_extracted_path = "/content/test"

In [ ]:
import os
# 훈련 데이터 압축 해제
with zipfile.ZipFile(train_zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(train_extracted_path)

# 테스트 데이터 압축 해제
with zipfile.ZipFile(test_zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(test_extracted_path)

tokenizer = T5TokenizerFast.from_pretrained("/content/drive/MyDrive/data/finetuning_model")
train_dataset = CustomDataset(os.path.join(train_extracted_path, 'train_original.json'), tokenizer)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)


In [ ]:
# 모델 설정
model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/data/finetuning_model")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

num_epochs = 3
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
import torch.optim as optim

# 학습 루프
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target_ids = batch['target_ids'].to(device)
        target_attention_mask = batch['target_attention_mask'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=target_ids,
            decoder_attention_mask=target_attention_mask
        )

        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        progress_bar.set_postfix({'Loss': total_loss / len(progress_bar)})

    torch.save(model.state_dict(), f'finetuned_model_epoch{epoch+1}.pt')


In [ ]:
# 파인 튜닝된 모델 저장
output_dir = "/content/drive/MyDrive/data/finetuning_model_sum"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
# 추론 예시
model.eval()
input_text = """이날 오전 9시 18분 현재 코스피는 전 거래일보다 17.47포인트(0.68%) 상승한 2,592.19다.

지수는 전장보다 15.17포인트(0.59%) 오른 2,589.89에 개장해 강세를 이어가고 있다.

유가증권시장에서 기관이 967억원을 순매수했다. 개인과 외국인은 각각 469억원, 512억원을 순매도했다.


지난 12일(현지시간) 미국 뉴욕증시에서 다우존스30산업평균지수(0.25%)와 스탠더드앤드푸어스(S&P) 500지수(0.74%), 기술주 중심의 나스닥지수(1.15%)가 일제히 상승했다.

S&P 500지수는 지난해 4월 8일 이후, 나스닥 지수는 지난해 4월 5일 이후 최고점이었다.

미국 6월 소비자물가지수(CPI)가 지난해 같은 달보다 3.0% 상승해 시장 예상치(3.1%)를 밑돌자 미국 연방준비제도(Fed·연준)의 긴축에 대한 부담이 줄어 시장이 환호한 것으로 보인다.

국내 증시는 미국 물가 지표에 호응하는 동시에 이날 장중 열리는 한국은행 금융통화위원회 통화정책 방향 회의 결과도 소화할 예정이다.

한지영 키움증권 연구원은 "미국의 물가 둔화를 한은도 확인한 만큼 금일 기준금리 동결이 유력하다"며 "어느 정도 수준으로 매파적인 동결을 할지가 시장에 영향을 미칠 수 있으므로 이창용 한은 총재의 발언에도 관심이 쏠릴 것"이라고 내다봤다.

시가총액 상위 10위권에서는 삼성전자(0.42%), LG에너지솔루션(0.56%), SK하이닉스(0.09%), 삼성바이오로직스(0.96%) 등 전 종목이 오르고 있다.

업종별로도 전기가스업(1.77%), 철강 및 금속(1.53%), 종이·목재(1.42%), 의약품(1.32%) 등을 비롯한 전 업종이 상승세다

같은 시각 코스닥지수도 전장보다 8.61포인트(0.98%) 높은 888.49다.

지수는 전장보다 5.33포인트(0.61%) 오른 885.21로 시작해 상승세를 지속하고 있다.


코스닥시장에서도 기관이 640억원을 순매수했다. 개인과 외국인은 각각 416억원, 187억원 매도 우위다.

시총 상위권에서는 셀트리온헬스케어(2.77%), JYP Ent.(1.04%), HLB(1.76%), 펄어비스(3.27%) 등이 강세다. 에코프로비엠(-1.97%)과 에코프로(-2.17%), 엘앤에프(-0.86%) 등은 하락세다."""
input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)
outputs = model.generate(input_ids=input_ids, max_length=300)

decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)